### Reconstructions_2DeteCT_dataset_Script

This script is used to produce the reconstructions for the sinogram data of the 2DeteCT dataset.

#### Initial imports

In [1]:
# This file contains a function for reconstructing sinogram data for the 2DeteCT dataset.

import astra
import imageio
import warnings
from typing import Any

import glob # Used for browsing through the directories.
import os # Used for creating directories.
import shutil # Later used for copying files.
import time # Used for keeping processing time.
from tqdm.notebook import tqdm_notebook # Used for tracking progress.
import NesterovGradient # Used for reconstruction method 'AGD'.
import ReadingSettings_2DeteCT # Used to read in the acquisition settings from a machine-readable .csv file

import numpy as np
from scipy.interpolate import interp1d # Used for detector shift via linear grid interpolation.
from scipy.sparse.linalg import lsqr # Used for reconstruction method 'LS'.
import matplotlib.pyplot as plt

In [2]:
# Data directories.
# We enter here some intrinsic details of the dataset needed for our reconstruction scripts.
# Set the variable "base_data_dir_str" to the path where the dataset is stored on your own workstation.
base_data_dir_str = '/export/scratch2/mbk/2DeteCT_tests/test1/'

# Set the variable "save_dir_str" to the path where you would like to store the reconstructions you compute.
save_dir_str = '/export/scratch2/mbk/2DeteCT_tests/test1_recons/' 

In [3]:
# User defined settings.

# Select the ID(s) of the slice(s) you want to reconstruct.
import random
slice_id = range(10,11) #random.sample(range(1, 6370+1), 50)
# Adjust this to your liking.

# Select which modes you want to reconstruct.
modes = range(3,4) # (1,2,3) #These are all modes available.

# Define whether you have a GPU for computations available.
use_GPU = True
gpuIndex = 0 # Set the index of the GPU card used.
astra.set_gpu_index(gpuIndex)

# Pre-processing parameters.
binning = 1 # Manual selection of detector pixel binning after acqusisition.
excludeLastPro = True # Exclude last projection angle which is often the same as the first one.
angSubSamp = 1 # Define a sub-sampling factor in angular direction.
# (all reference reconstructions are computed with full angular resolution).
maxAng = 360 # Maximal angle in degrees - for reconstructions with limited angle (standard: 360).

# Correction profiles.
# The detector is slightly shifted with respect to the ASTRA geometry specified.
# Furthermore, the detector has been changed shortly before 20220531 (between slices 2830 and 2831).
# The full correction profiles can be found below.
corr_profiles = dict()
corr_profiles['20220407_RvL'] = {'det_tan': 24.19, 'src_ort': -5.67, 'axs_tan': -0.5244, 'det_roll': -0.015}
corr_profiles['20220531_RvL'] = {'det_tan': 24.4203, 'src_ort': -6.2281, 'axs_tan': -0.5010, 'det_roll': -0.262}

# This array contains the simplified horizontal correction shift for both geometries.
corr = np.array([2.75, 1.00]) # Found to be the optimal shifts for before and after detector exchange.

# File names in dataset structure.
sino_name = 'sinogram.tif'
dark_name = 'dark.tif'
flat_name = ('flat1.tif', 'flat2.tif')
slcs_name ="slice{:05}"

# Reference information.
settings = ReadingSettings_2DeteCT.SettingsFile_2DeteCT('./Mode3_settings.csv') # Read in settings file (the mode does not play a role for the recons)
sino_dims = (settings.get_parameter('ProjectionsNumber'),settings.get_parameter('ProjectionLength')) # Dimensions of the full sinograms.
detPix = settings.get_parameter('DetPixSize') # Physical size of one detector pixel in mm.
# Early OOD scans: 5521 - 5870 
# Late OOD scans: 5871 - 6370

# Reconstruction parameters.
recMeth = 'AGD' # Specify which reconstruction algorithm you want to use.
recMeths = ['AGD'] # ['FBP', 'LS', 'NNLS', 'SIRT', 'SART', 'CGLS', 'AGD'] # List of reconstruction algorithms to iterate over.
recSz = (4096,4096) # Used reconsttuction area to create as little model-inherent artifacts within the FOV.
outSz = (2048,2048) # Output size before downscaling corresponding to the FOV.
maxIter = 100 # Specify the maximal iteration number.

# Visualization of results.
visuals = False

In [4]:
# Helper functions.

def norm(u):
    # L2 norm of u interpreted as a vector.
    return np.sqrt(np.sum(u**2))

def power_iteration(A, num_simulations):
        # Ideally choose a random vector
        # To decrease the chance that our vector
        # Is orthogonal to the eigenvector.
        b_k = np.random.rand(A.shape[1])
        b_k1_norm = 1

        print('running power iteration to determine step size', flush=True)
        for i in range(num_simulations):

            # Calculate the matrix-by-vector product Ab.
            b_k1 = A.T*A*b_k

            # Calculate the norm.
            b_k1_norm = np.linalg.norm(b_k1)

            # Renormalize the vector.
            b_k = b_k1 / b_k1_norm
        print('found step size to be:', 1./b_k1_norm , flush=True)
        return b_k1_norm

In [5]:
# Keep track of the processing time per reconstruction job.
t = time.time()
print('Starting reconstruction job...', flush=True)

# Helper variables.
residua = np.zeros((len(slice_id),len(modes),1))
slc_counter = 0

# Iterate over all methods, all slices, and all modes.
for i_meth in tqdm_notebook(range(len(recMeths)), desc = 'Loop over all methods'):
    recMeth = recMeths[i_meth]
    
    for i_slc in tqdm_notebook(slice_id, desc = 'Loop over all desired slices in the dataset'):

        for i_mode in tqdm_notebook(modes, desc = 'Loop over all desired acquisition modes'):

            # Load and pre-process data.

            # Get the current path for respective slice and mode within the dataset structure.
            current_path = base_data_dir_str + slcs_name.format(i_slc) + '/mode{}/'.format(i_mode)

            # load flat-field and dark-fields.
            # There are two flat-field images (taken before and after the acquisition of ten slices),
            # we simply average them.
            dark = imageio.imread(glob.glob(current_path + dark_name)[0]) 
            flat1 = imageio.imread(glob.glob(current_path + flat_name[0])[0])
            flat2 = imageio.imread(glob.glob(current_path + flat_name[1])[0])
            flat = np.mean(np.array([ flat1, flat2 ]), axis=0 )
            
            # Read in the sinogram.
            sinogram = imageio.imread(glob.glob(current_path + sino_name)[0])
            sinogram =  np.ascontiguousarray(sinogram)

            # Subtract the dark field, devide by the flat field,
            # and take the negative log to linearize the data according to the Beer-Lambert law.
            data = sinogram - dark
            data = data/(flat-dark)
            
            # Exclude last projection if desired.
            if excludeLastPro:
                data = data[0:-1,:]

            # Create detector shift via linear grid interpolation.
            if i_slc in range(1,2830+1) or i_slc in range(5521,5870+1):
                detShift = corr[0] * detPix
            else:
                detShift = corr[1] * detPix
            
            detGrid = np.arange(0,1912) * detPix
            detGridShifted = detGrid + detShift
            detShiftCorr = interp1d(detGrid, data, kind='linear', bounds_error=False, fill_value='extrapolate')
            data = detShiftCorr(detGridShifted)

            # Clip the data on the lower end to 1e-6 to avoid division by zero in next step.
            data = data.clip(1e-6, None)
            print("Values have been clipped from [", np.min(data), ",", np.max(data),"] to [1e-6,None]")

            # Take negative log.
            data = np.log(data)
            data = np.negative(data)
            data = np.ascontiguousarray(data)

            # Create array that stores the used projection angles.
            angles = np.linspace(0,2*np.pi, settings.get_parameter('ProjectionsNumber'))
            
            # Apply exclusion of last projection if desired.
            if excludeLastPro:
                angles = angles[0:-1]
                print('Excluded last projection.')
            
            # Apply angular subsampling.
            data = data[0::angSubSamp,:]
            angles = angles[0::angSubSamp]
            angInd = np.where(angles<=(maxAng/180*np.pi))
            angles = angles[angInd]
            data = data[:(angInd[-1][-1]+1),:]

            print('Data shape:', data.shape)
            print('Length angles:', len(angles))
            
            print('Loading and pre-processing done', flush=True)

            
            print('Computing reconstruction for slice', i_slc, '...', flush=True)

            # Create ASTRA objects for reconstruction.
            detSubSamp = 1
            binning = 1
            detPixSz = detSubSamp * binning * detPix
            SOD = settings.get_parameter('SOD') # Source-Object-Distance for the scanning geometry of the 2DeteCT dataset.
            SDD = settings.get_parameter('SDD') # Source-Detector-Distance for the scanning geometry of the 2DeteCT dataset.

            # Scale factor calculation.
            # ASTRA assumes that the voxel size is 1mm.
            # For this to be true we need to calculate a scale factor for the geometry.
            # This can be done by first calculating the 'true voxel size' via the intersect theorem
            # and then scaling the geometry accordingly.

            # Physical width of the detector.
            nPix = settings.get_parameter('ProjectionLength')
            det_width = detPixSz * nPix

            # Physical width of the field of view in the measurement plane via intersect theorem.
            FOV_width = det_width * SOD/SDD
            print('Physical width of FOV (in mm):', FOV_width)

            # True voxel size with a given number of voxels to be used.
            nVox = 2048
            voxSz = FOV_width / nVox
            print('True voxel size (in mm) for', nVox, 'voxels to be used:', voxSz)

            # Scaling the geometry accordingly.
            scaleFactor = 1./voxSz
            print('Self-calculated scale factor:', scaleFactor)
            SDD = SDD * scaleFactor
            SOD = SOD * scaleFactor
            detPixSz = detPixSz * scaleFactor

            # Create ASTRA objects.
            projGeo = astra.create_proj_geom('fanflat', detPixSz, 1912, angles, SOD, SDD - SOD)
            volGeo = astra.create_vol_geom(recSz[0], recSz[1])
            recID = astra.data2d.create('-vol', volGeo)
            sinoID = astra.data2d.create('-sino', projGeo, data)
            projID   = astra.create_projector('cuda', projGeo, volGeo)
            A = astra.OpTomo(projID)

            if recMeth == 'FBP':
                # Create ASTRA configuration.
                if use_GPU:
                    alg_name = recMeth + '_CUDA'
                    cfg = astra.astra_dict(alg_name)
                else:
                    cfg = astra.astra_dict(recMeth)
                    proj_id = astra.create_projector('line_fanflat', projGeo, volGeo)
                    cfg['ProjectorId'] = proj_id

                cfg['ReconstructionDataId'] = recID
                cfg['ProjectionDataId'] = sinoID
                cfg['option'] = {'FilterType': 'Ram-Lak', 'MinConstraint': 0.}

                # Create and run algorithm.
                algID = astra.algorithm.create(cfg)
                iterations = maxIter
                astra.algorithm.run(algID, iterations)

                # Receive reconstruction.
                rec = astra.data2d.get(recID)
                rec = np.maximum(rec,0)
                
                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)

                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                    +'_FBP'+'_iter{}'.format(iterations)+'.tif'), (rec_cut.astype(np.float32)).reshape(outSz))


            elif recMeth == 'LS':
                # Use lsqr method from scipy.sparse.linalg to calculate the reconstruction.
                rec, istop, itn, residua[slc_counter, i_mode-1, 0]  = lsqr(A,data.flatten(), iter_lim=maxIter)[:4]
                print('Iteration stopped because of', istop, 'after', itn, 'iterations with residual',residua[0,i_mode-1,0])
                
                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec.reshape(recSz)[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)
                
                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)+'_LS'+
                                    '_iter{}'.format(itn)+".tif"),(rec_cut.astype(np.float32)).reshape(outSz))


            elif recMeth == 'NNLS':
                
                # Calculate Lipschitz constant via power iteration.
                lipschitz = power_iteration(A,10)
                tau = 1e-6 # Specify step-size yourself or use 1./lipschitz.
                tau = 1./lipschitz

                u = np.zeros(A.vshape) # The unknown reconstructed image.
                normF       = norm(data)

                # Compute D(f, Au) and grad D(f, Au) at u = 0 (--> Au = 0).
                residuum     = - data 
                relResNorm   = 1.0
                Du           = 0.5 * np.sum(residuum**2)
                gradDu       = A.BP(residuum)

                # Computing the variational energy J(u) and its gradient.
                Ju, gradJu   = Du, gradDu            

                # Projected gradient descent iteration.
                for iter in tqdm_notebook(range(maxIter+1)):

                    # Update u by a gradient step followed by a projection.
                    uNew = u - tau * gradJu

                    # Activate non-negativity constraint.
                    uNew[uNew < 0] = 0

                    # Compute D(f, Au) and grad D(f, Au) at uNew.
                    residuumNew     = A.FP(uNew) - data
                    relResNormNew   = norm(residuumNew) / normF # To make the residuum comparable to fitted term f.
                    DuNew           = 0.5 * np.sum(residuumNew**2)
                    gradDuNew       = A.BP(residuumNew)

                    # Compute variational energy J(u) and its gradient grad J(u) at uNew.
                    JuNew, gradJuNew = DuNew, gradDuNew

                    if JuNew > Ju:

                        # Discard update step and decrease step size.  
                        tau      = 0.9 * tau
                        changeU, changeJu = 0.0, 0.0

                    else:
                        # Accept new c.
                        # For displaying purposes compute the rel. change of u and J(u).
                        changeU       = norm(uNew - u)   / norm(uNew)
                        changeJu      = (Ju - JuNew) / Ju

                        # Update the variables with new values.
                        u, Ju, gradJu = uNew, JuNew, gradJuNew

                    # Display some output now and then.
                    if not(iter % 100):
                        print("it: {:-6}; Ju: {:1.6e} (rel change: {:1.3e}); rel change u: {:1.3e}; tau: {:1.3e}".format(
                            iter, Ju, changeJu, changeU, tau))
                    
                    # Cut the reconstruction to the desired area of (2048,2048).
                    if iter == maxIter:
                        rec_cut = u[1023:3071,1023:3071]
                        print('Shape of cut out reconstruction area:', rec_cut.shape)
                        
                        # Save reconstruction.
                        imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                            +'_NNLS_iter'+str(iter)+'.tif'),(rec_cut.astype(np.float32)).reshape(outSz))

            elif recMeth == 'SIRT':
                
                # Create configuration.
                if use_GPU:
                    alg_name = recMeth + '_CUDA'
                    cfg = astra.astra_dict(alg_name)
                else:
                    cfg = astra.astra_dict(recMeth)
                    proj_id = astra.create_projector('fanflat', projGeo, volGeo)
                    cfg['ProjectorId'] = proj_id

                cfg['ReconstructionDataId'] = recID
                cfg['ProjectionDataId'] = sinoID
                cfg['option'] = {'MinConstraint': 0}

                # Create and run algorithm.
                algID = astra.algorithm.create(cfg)
                iterations = maxIter
                astra.algorithm.run(algID, iterations)

                # Receive reconstruction.
                rec = astra.data2d.get(recID)
                rec = np.maximum(rec,0) 

                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)

                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                    +'_SIRT_iter'+str(iterations)+'.tif'),(rec_cut.astype(np.float32)).reshape(outSz))

            elif recMeth == 'SART':
                
                # Create configuration.
                if use_GPU:
                    alg_name = recMeth + '_CUDA'
                    cfg = astra.astra_dict(alg_name)
                else:
                    cfg = astra.astra_dict(recMeth)
                    proj_id = astra.create_projector('fanflat', projGeo, volGeo)
                    cfg['ProjectorId'] = proj_id

                cfg['ReconstructionDataId'] = recID
                cfg['ProjectionDataId'] = sinoID
                cfg['option'] = {'MinConstraint': 0}

                # Create and run algorithm.
                algID = astra.algorithm.create(cfg)
                iterations = maxIter
                astra.algorithm.run(algID, iterations)

                # Receive reconstruction.
                rec = astra.data2d.get(recID)
                rec = np.maximum(rec,0) 
                
                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)

                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                    +'_SART_iter'+str(iterations)+'.tif'),(rec_cut.astype(np.float32)).reshape(outSz))

            elif recMeth == 'CGLS':
                
                # Create configuration.
                if use_GPU:
                    alg_name = recMeth + '_CUDA'
                    cfg = astra.astra_dict(alg_name)
                else:
                    cfg = astra.astra_dict(recMeth)
                    proj_id = astra.create_projector('fanflat', projGeo, volGeo)
                    cfg['ProjectorId'] = proj_id

                cfg['ReconstructionDataId'] = recID
                cfg['ProjectionDataId'] = sinoID

                # Create and run algorithm.
                algID = astra.algorithm.create(cfg)
                iterations = maxIter
                astra.algorithm.run(algID, iterations)

                # Receive reconstruction.
                rec = astra.data2d.get(recID)
                rec = np.maximum(rec,0) 

                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)
                
                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                        +'_CGLS_iter'+str(iterations)+'.tif'),(rec_cut.astype(np.float32)).reshape(outSz))

            
            elif recMeth == 'AGD':
                
                # Create an ASTRA configuration using a registered plugin.
                # This configuration dictionary setups an algorithm,
                # a projection and a volume geometry and returns
                # an ASTRA algorithm, which can be run on its own.
                
                astra.plugin.register(NesterovGradient.AcceleratedGradientPlugin)
                proj_id = astra.create_projector('cuda', projGeo, volGeo)
                cfg_agd = astra.astra_dict('AGD-PLUGIN')
                cfg_agd['ReconstructionDataId'] = recID
                cfg_agd['ProjectionDataId'] = sinoID
                cfg_agd['ProjectorId'] = proj_id
                cfg_agd['option'] = {}
                cfg_agd['option']['MinConstraint'] = 0
                
                # Create and run algorithm.
                algID = astra.algorithm.create(cfg_agd)
                iterations = maxIter
                astra.algorithm.run(algID, iterations)

                # Receive reconstruction.
                rec = astra.data2d.get(recID)
                rec = np.maximum(rec,0)
                
                # Cut the reconstruction to the desired area of (2048,2048).
                rec_cut = rec[1023:3071,1023:3071]
                print('Shape of cut out reconstruction area:', rec_cut.shape)
                
                # Save reconstruction.
                imageio.imwrite(str(save_dir_str+"recon_"+ slcs_name.format(i_slc) + '_mode{}'.format(i_mode)
                                                +'_AGD_iter'+str(iterations)+'.tif'),(rec_cut.astype(np.float32)).reshape(outSz))
            
            if visuals:                
                plt.imshow(rec_cut, cmap='gray')                

            # Clean up.
            astra.algorithm.delete(algID)
            astra.data2d.delete(recID)
            astra.data2d.delete(sinoID)
            
            if not use_GPU:
                astra.projector.delete(proj_id)
        
        # Update counter for LS to store residua.
        if recMeth == 'LS':
            slc_counter +=1

print(np.round_(time.time() - t, 3), 'sec elapsed for reconstructing', len(slice_id), 'slices.')

Starting reconstruction job...


Loop over all methods:   0%|          | 0/1 [00:00<?, ?it/s]

Loop over all desired slices in the dataset:   0%|          | 0/1 [00:00<?, ?it/s]

Loop over all desired acquisition modes:   0%|          | 0/1 [00:00<?, ?it/s]

Values have been clipped from [ 0.06209030751874151 , 1.0728267484959892 ] to [1e-6,None]
Excluded last projection.
Data shape: (3600, 1912)
Length angles: 3600
Loading and pre-processing done
Computing reconstruction for slice 10 ...
Physical width of FOV (in mm): 116.52814274682939
True voxel size (in mm) for 2048 voxels to be used: 0.05689850720060029
Self-calculated scale factor: 17.575153535652863
running power iteration to determine step size
plugin initialized.
running 100 iterations of Accelerated Gradient plugin.
iteration 0 / 100
iteration 10 / 100
iteration 20 / 100
iteration 30 / 100
iteration 40 / 100
iteration 50 / 100
iteration 60 / 100
iteration 70 / 100
iteration 80 / 100
iteration 90 / 100
Shape of cut out reconstruction area: (2048, 2048)
272.863 sec elapsed for reconstructing 1 slices.
